python 3.7 and pytorch 1.8

# pytorch 搭建简单MLP模型

- 目的：
- 输入：
- 输出：
- 主要步骤：
1.

In [ ]:
# 0. 初始化全局变量
INPUT_DIM = 128  # 输入数据的大小为 128
HIDDEN_DIM = 216  # 隐层大小为 216，或者叫latent_dim
OUTPUT_DIM = 3  # 输出大小为 3


In [ ]:
# 1. 建立模型
import torch.nn as nn
# F 相当于tf中的layers，具有多种层类型
import torch.nn.functional as F

class Net(nn.Module):
    # 模型定义，继承 Module 类
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Net, self).__init__()
        # 隐藏层，参数大小：input_dim * hidden_dim
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # 输出层，参数大小：hidden_dim * output_dim
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)  # 输入 x 经过隐藏层线性运算
        x = F.relu(x)  # 经过激活函数 ReLU 运算
        x = F.relu(self.fc2(x))  # 经过输出层线性及激活运算，最后输出
        return x

In [ ]:
# 模型初始化
my_first_model = Net(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(my_first_model)  # 打印显示模型结构

# 训练模型
import torch
# 生成模拟数据
N = 10  # 假设一批数据为 10 个
x = torch.randn(N, INPUT_DIM)  # 数据大小: [10，128]

In [ ]:
output = my_first_model(x)  # 不需要调用 forward（）
print(output)
print(output.shape)
